In [ ]:
"""
The purpose is to create a classifier to classify the articles into "flood" and "not-flood"
Using the title of the article.
"""

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import nltk
import re

In [ ]:
# Read one of the .csv file prepared from article scraping task # 4
df = pd.read_csv('FL-2019-000022-IRN.csv', index_col = 1)

In [ ]:
#based on what @Luca Pietrobon prepared:

df = df.dropna(axis = 1, how = 'all').rename(columns={c: 'Categories' for c in ['category', 'Category']})

df = df.drop(columns = [c for c in df.columns if c.startswith('Unnamed')]).rename(columns = {c: c.lower().replace (' ', '_') for c in df.columns})

df['label_isFlood'] = 1 *df.categories.str.contains('flood')
df['text'] = df.article_title.str.split(r'\s-|\|\s').str[0]


In [ ]:
#shuffle the DF
df = df.sample(frac = 1)

#Split the DF
y = df.pop('label_isFlood')
X = df

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)
               

In [ ]:
# Import the SnowballStemmer to perform stemming
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# Define a function to perform both stemming and tokenization
def tokenize_and_stem(text):
    
    # Tokenize by sentence, then by word
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    
    # Filter out raw tokens to remove noise
    filtered_tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]
    
    
    # Stem the filtered_tokens
    stems = [stemmer.stem(t) for t in filtered_tokens]
    
    return stems


In [ ]:
# Import TfidfVectorizer to create TF-IDF vectors
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate TfidfVectorizer object with stopwords and tokenizer
# parameters for efficient processing of text
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=5,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem,
                                 ngram_range=(1,3))

    
# Fit and transform the tfidf_vectorizer with the "Title (text column in DF)" 
# to create a vector representation 
tfidf_matrix = tfidf_vectorizer.fit_transform([x for x in X_train['text']])

tfidf_matrix2 = tfidf_vectorizer.fit_transform([x for x in X_test['text']])

In [ ]:
#Using Naive Bayes to do the prediction

from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(tfidf_matrix.toarray(),y_train)

y_hat = (clf.predict(tfidf_matrix2.toarray()))

clf.score(tfidf_matrix.toarray(),y_train)
clf.score(tfidf_matrix2.toarray(),y_test)

print(confusion_matrix(y_test, y_hat))